## Preprocesamiento 

### Carga y preprocesamiento inicial

Primero empezamos por hacer la lectura inicial al conjunto de datos 'german.data' que según la fuente contiene la información en datos de tipo categórico y numérico:

In [1]:
import pandas as pd
import os 

ruta_archivo = os.path.join('..', 'data', 'raw', 'german.data')

g_credit_data = pd.read_csv(ruta_archivo, sep=' ', header=None, skipinitialspace=True)

In [2]:
g_credit_data

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


Podemos ver que el conjunto de datos en su versión inicial no está muy legible de cara a un EDA, por lo tanto tendremos que modificar el conjunto de datos para que tenga sus atributos legibles y nombres de columnas
identificados. Para hacer eso vamos a utilizar una función que se encarga de mapear los valores a su versión legible basándose en la información de la [fuente oficial de los datos](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data):

In [3]:
from tp_final_fairness_ML.utils.preprocessing import mapear_german_credit_data

g_credit_data = mapear_german_credit_data(g_credit_data)
g_credit_data.head()

,checking_account,duration,credit_history,purpose,credit_amount,savings_account,employment_since,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,target
0,< 0 DM,6,critical account / other bank,radio/TV,1169,unknown / no account,>= 7 years,4,"male, single",none,...,real estate,67,none,own,2,skilled / official,1,"yes, registered",yes,1
1,0 <= ... < 200 DM,48,paid duly till now,radio/TV,5951,< 100 DM,1 <= ... < 4 years,2,"female, div/sep/married",none,...,real estate,22,none,own,1,skilled / official,1,none,yes,2
2,no account,12,critical account / other bank,education,2096,< 100 DM,4 <= ... < 7 years,2,"male, single",none,...,real estate,49,none,own,1,unskilled - resident,2,none,yes,1
3,< 0 DM,42,paid duly till now,furniture/equipment,7882,< 100 DM,4 <= ... < 7 years,2,"male, single",guarantor,...,building society / insurance,45,none,for free,1,skilled / official,2,none,yes,1
4,< 0 DM,24,delay in past,car (new),4870,< 100 DM,1 <= ... < 4 years,3,"male, single",none,...,unknown / no property,53,none,for free,2,skilled / official,2,none,yes,2


Ya con un conjunto de datos con valores legibles nos queda como último paso, pensando en el análisis exploratorio, el crear una columna adicional que represente a la variable objetivo pero de forma categórica. La idea sería la siguiente: 

* Generamos una nueva columna llamada 'target_label'.

* Esta columna contendrá como posibles valores 'low_risk' y 'high_risk' que corresponden al 1 y al 2 respectivamente.

Esto logrará que el análisis exploratorio sea más ameno y los gráficos generados más fáciles de interpretar.

In [4]:
g_credit_data['target_label'] = g_credit_data['target'].map({1: 'low_risk', 2: 'high_risk'})

Verificamos que los cambios se hayan aplicado correctamente:

In [5]:
g_credit_data[['target', 'target_label']].head(10)

,target,target_label
0,1,low_risk
1,2,high_risk
2,1,low_risk
3,1,low_risk
4,2,high_risk
5,1,low_risk
6,1,low_risk
7,1,low_risk
8,1,low_risk
9,2,high_risk


### Verificación de consistencia de los datos

Antes de proceder con el EDA siempre es buena práctica mirar si hay valores faltantes en el conjunto de datos, que los tipos sean los correctos y que las columnas sean las esperadas:

In [6]:
g_credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   checking_account         1000 non-null   object
 1   duration                 1000 non-null   int64 
 2   credit_history           1000 non-null   object
 3   purpose                  1000 non-null   object
 4   credit_amount            1000 non-null   int64 
 5   savings_account          1000 non-null   object
 6   employment_since         1000 non-null   object
 7   installment_rate         1000 non-null   int64 
 8   personal_status_sex      1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  residence_since          1000 non-null   int64 
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housing                  1000 non-null   

Podemos ver que todo está bien, tenemos 1000 registros y todas las columnas especificadas en la fuente de los datos están presentes. Lo único que queda hacer es cambiar los tipos de datos de las
columnas categóricas de 'object' a 'category' para hacer un EDA adecuadamente:

In [7]:
from tp_final_fairness_ML.utils.preprocessing import convertir_object_a_categ

g_credit_data = convertir_object_a_categ(g_credit_data)

g_credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   checking_account         1000 non-null   category
 1   duration                 1000 non-null   int64   
 2   credit_history           1000 non-null   category
 3   purpose                  1000 non-null   category
 4   credit_amount            1000 non-null   int64   
 5   savings_account          1000 non-null   category
 6   employment_since         1000 non-null   category
 7   installment_rate         1000 non-null   int64   
 8   personal_status_sex      1000 non-null   category
 9   other_debtors            1000 non-null   category
 10  residence_since          1000 non-null   int64   
 11  property                 1000 non-null   category
 12  age                      1000 non-null   int64   
 13  other_installment_plans  1000 non-null   category
 14  housing  

Ahora sí que todo está bien podemos proceder con el EDA.